## Productos
- `codigo`: 4 digitos
- `nombre`: 1 a 100 caracteres
- `precio`: 10 a 10000
- `tipo`: 0 a 20 caracteres
- `cantidad`: 0 a 100 
- `valorTotal`: cantidad * precio

In [12]:
class Producto:
    def __init__(self, codigo, nombre, precio, tipo, cantidad):
        
        self.codigo = codigo
        self.nombre = nombre
        self.precio = precio
        self.tipo = tipo
        self.cantidad = cantidad

    @property
    def precio(self):
        return self._precio
        
    @precio.setter
    def precio(self, valor):
        if valor < 10 or valor > 10000:
            print(f"El precio {valor} ingresado es invalido")
        else:    
            self._precio = valor
        
    @property
    def cantidad(self):
        return self._cantidad
        
    @cantidad.setter
    def cantidad(self, valor):
        if valor <= 0:
            print(f"La cantidad {valor} ingresada es invalida")
        else:    
            self._cantidad = valor

    def valorTotal(self):
        return self.precio * self.cantidad


               

In [13]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba de la clase Producto

p1 = Producto('0001', 'Coca Cola', 1500, 'gaseosa', 10)

assert p1.codigo == '0001'
assert p1.nombre == 'Coca Cola'
assert p1.precio == 1500

# Calcula el valor total 
assert p1.valorTotal() == 15000 

# Asegura que los valores de precio y cantidad sean validos
p1.precio = -200
assert p1.precio == 1500    # Rechaza el valor negativo

p1.precio = 2000 
assert p1.precio == 2000

p1.cantidad = -5
assert p1.cantidad == 10    # Rechaza el valor negativo

p1.cantidad = 50
assert p1.cantidad == 50
assert p1.valorTotal() == 100000

print("Prueba pasada exitosamente!")

El precio -200 ingresado es invalido
La cantidad -5 ingresada es invalida
Prueba pasada exitosamente!


# Ofertas
Debe permitir aplicar oferctas a codigos espeficicos de productos y a tipos de productos
- `descripcion`: 1 a 100 caracteres
- `codigos`: lista de codigos de productos
- `tipos`: lista de tipo de producto
- `esAplicable(producto, cantidad)`: retorna si la oferta es aplicable a ese producto
- `aplicar(producto, cantidad)`: retorna el precio final del producto con la oferta aplicada


In [14]:

class Oferta:
    def __init__(self, codigos = None, tipos = None):
        self.codigos = codigos or []
        self.tipos = tipos or []
    
    def esAplicable(self, producto, cantidad):

        codigo_valido = producto.codigo in self.codigos
        tipo_valido = producto.tipo in self.tipos
        return codigo_valido or tipo_valido

class OfertaDescuento(Oferta):
    def __init__(self, descuento,*args, **kwargs):
        super().__init__(*args, **kwargs)
        self.descuento = descuento
        

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            total = producto.precio * cantidad
            descuento_aplicado = total * (self.descuento / 100)
            return descuento_aplicado
        return 0

    def aplicar(self, producto, cantidad):
        descuento = self.calcularDescuento(producto, cantidad)
        return (producto.precio * cantidad) - descuento
    
class Oferta2x1(Oferta):
    def __init__(self, codigos = None, tipos = None,  *args, **kwargs):
        super().__init__(codigos, tipos, *args, **kwargs)

    def calcularDescuento(self, producto, cantidad):
        if self.esAplicable(producto, cantidad):
            if cantidad < 2:
                return 0
            return producto.precio * (cantidad // 2)
        return 0
    
    def aplicar(self, producto, cantidad):
        return self.calcularDescuento(producto, cantidad)
        
    

In [15]:
## NO MODIFICAR ESTE CODIGO ##

p1 = Producto('1234', 'Coca Cola', 1000, 'gaseosa', 10)
p2 = Producto('1235', 'Oreo',      2300, 'galleta', 10)

o10d = OfertaDescuento(10, codigos=['1234'])
assert o10d.calcularDescuento(p1, 10) == 1000 
assert o10d.calcularDescuento(p1, 1) == 100

assert o10d.calcularDescuento(p2, 10) == 0

o2x1 = Oferta2x1(tipos=['galleta'])
assert o2x1.calcularDescuento(p1, 10) == 0

assert o2x1.calcularDescuento(p2, 1) == 0
assert o2x1.calcularDescuento(p2, 2) == 2300
assert o2x1.calcularDescuento(p2, 3) == 2300
assert o2x1.calcularDescuento(p2, 4) == 4600
assert o2x1.calcularDescuento(p2, 5) == 4600

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Catalogo
- `leer(archivo) `    : Carga los productos desde el archivo
- `guardar(archivo)`  : Guarda los productos en el archivo
- `agregar(producto)` : Agrega un producto al catalogo
- `buscar(codigo)`    : Busca un producto por codigo o None si no existe
- `registrarOferta(oferta)`  : Registra una oferta
- `buscarOferta(producto, cantidad)`: Busca una oferta por codigo o None si no existe
- `calcularDescuento(producto, cantidad)`: Calcula el descuento de una oferta
- `cantidadProductos`: Retorna la cantidad de productos en el catalogo
- `cantidadUnidades`: Retorna la cantidad de unidades en el catalogo
- `valorTotal`: retorna el valor total del catalogo sin descuentos
- `informe()`: retorna un string con el informe del catalogo 

In [16]:
import csv
class Catalogo:
    def __init__(self) :
        self.productos = []
        self.ofertas = []

    def agregar(self, *productos):
        for producto in productos:
            self.productos.append(producto)

    def buscar(self, codigo):
        for producto in self.productos:
            if producto.codigo == codigo:
                return producto
        return None
    
    def registrarOferta(self, oferta):
        self.ofertas.append(oferta)

    def buscarOfertas(self, producto):
        aplicables = []
        for oferta in self.ofertas:
            if oferta.aplica(producto):
                aplicables.append(oferta)
        return aplicables

    def buscarOferta(self, producto, cantidad):
        for oferta in self.ofertas:
            if oferta.esAplicable(producto, cantidad):
                return oferta
        return None

    def calcularDescuento(self, producto, cantidad):
        mejor_descuento = 0
        for oferta in self.ofertas:
            if oferta.esAplicable(producto, cantidad):
                descuento = oferta.calcularDescuento(producto, cantidad)
                if descuento > mejor_descuento:
                    mejor_descuento = descuento
        return mejor_descuento
    
    
    @property

    def cantidadProductos(self):
        return len(self.productos)
    
    @property

    def cantidadUnidades(self):
        return sum(producto.cantidad for producto in self.productos) 
    
    @property
    
    def valorTotal(self):
        return sum(p.valorTotal() for p in self.productos)
    
    def guardar (self, archivo):
        with open(archivo, mode='w', newline='', encoding='utf-8') as file:
            escritor = csv.writer(file)
            escritor.writerow(['codigo', 'nombre', 'precio', 'tipo', 'cantidad'])
            for producto in self.productos:
                escritor.writerow([producto.codigo, producto.nombre, producto.precio, producto.tipo, producto.cantidad])


    def leer(pArchivo):
        catalogo = Catalogo()

        with open(pArchivo, mode='r', newline='', encoding='utf-8') as file:
            lector = csv.reader(file)
            next(lector)

            for row in lector:
                if len(row) == 5:
                    codigo, nombre, precio, tipo, cantidad = row
                    producto = Producto(codigo, nombre, float(precio), tipo, int(cantidad))
                    catalogo.agregar(producto)
        return catalogo

    def vender(self, producto, cantidad):
        p = self.buscar(producto.codigo)
        if p and p.cantidad >= cantidad:
            p.cantidad -= cantidad
        else:
            print("no hay stock")

    def informe(self):
        total = self.valorTotal  # Esta línea está correcta ya que valorTotal es una propiedad
        tipos = set(p.tipo for p in self.productos)

        if self.cantidadProductos > 0:
            precio_promedio = total / self.cantidadProductos

        else:
            precio_promedio = 0

        Oferta_str = ', '.join([type(oferta).__name__.replace('Oferta', 'Oferta ') for oferta in self.ofertas])

        return (f"Cantidad de productos: {self.cantidadProductos}\n"
                f"Cantidad de unidades: {self.cantidadUnidades}\n"
                f"Precio Promedio: {precio_promedio:.2f}\n"
                f"Valor total: {total}\n"
                f"Tipos de productos: {', '.join(tipos)}\n"
                f"Ofertas: {Oferta_str}")
    
    
    

In [17]:
## NO MODIFIQUE ESTE CODIGO ##

# Prueba del catálogo 

catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)

## Agregar productos al catalogo 
catalogo.agregar(p1)
catalogo.agregar(p2)
catalogo.agregar(p3)
catalogo.agregar(p4)

assert catalogo.cantidadProductos == 4
assert catalogo.cantidadUnidades == 100

assert catalogo.valorTotal == 167000

## Calcular descuentos segun las ofertas registradas
assert catalogo.calcularDescuento(p1, 5) == 0
assert catalogo.calcularDescuento(p2, 5) == 0

# Ofertas no acumulables 
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

assert catalogo.calcularDescuento(p1, 5) == 750
assert catalogo.calcularDescuento(p2, 5) == 0
assert catalogo.calcularDescuento(p3, 5) == 2400

assert catalogo.valorTotal == 167000.0
catalogo.guardar('catalogo-prueba.csv') ## Guardar datos antes de vender

# Vender afecta la cantidad de unidades y el valor total
catalogo.vender(p3, 3)   

# Verificar que el informe se genere correctamente

informe = catalogo.informe()
assert "Cantidad de productos: " in informe
assert "Cantidad de unidades: " in informe
assert "Precio Promedio: " in informe
assert "Valor total: " in informe
assert "Tipos de productos: " in informe
assert "gaseosa" in informe
assert "galleta" in informe
assert "Ofertas:" in informe 
assert "Oferta 2x1" in informe
assert catalogo.cantidadUnidades == 97
assert catalogo.valorTotal == 163400

# Buscar por código
assert catalogo.buscar('0001') == p1
assert catalogo.buscar('0002') == p2
assert catalogo.buscar('0099') is None 

# Recuperar los datos guardados  
c2 = Catalogo.leer('catalogo-prueba.csv')
assert c2.cantidadProductos == 4
assert c2.cantidadUnidades == 100

# Valor antes de guardar
assert c2.valorTotal == 167000.0

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!


# Cliente
- `nombre`: 1 a 100 caracteres
- `cuit`: 13 digitos (formato XX-XXXXXXXX-X)

In [18]:
class Cliente: 
    def __init__(self, nombre, cuit):
        self.nombre = nombre
        self.cuit = cuit

    @property
    def nombre(self):
        return self._nombre
    
    @nombre.setter
    def nombre (self, valor):
        if 1 <= len(valor) <= 100:
            self._nombre = valor

        else:
            print(f"El nombre {valor} es invalido")

    @property
    def cuit(self):
        return self._cuit
    
    @cuit.setter
    def cuit(self, valor):
        if self.validar_cuit(valor):
            self._cuit = valor
        else:
            print(f"El cuit {valor} es invalido")

    @staticmethod
    def validar_cuit(cuit):
        if len(cuit) != 13:
            return False
        if cuit[2] != '-' or cuit[11] != '-':
            return False
        if not (cuit[:2].isdigit() and cuit[3:11].isdigit() and cuit[12].isdigit()):
            return False
        return True

In [19]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Cliente #

c1 = Cliente('Juan Perez', '20-12345678-1')

assert c1.nombre == 'Juan Perez'
assert c1.cuit   == '20-12345678-1'

c1.nombre = ''
assert c1.nombre == 'Juan Perez' # Rechaza el valor vacio

c1.nombre = 'Juana Perez'        # Acepta el nuevo valor
assert c1.nombre == 'Juana Perez'

c1.cuit = '1234567890123'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

c1.cuit = 'CC-12345678-1'
assert c1.cuit == '20-12345678-1' # Rechaza el valor incorrecto

print("Prueba pasada exitosamente!")

El nombre  es invalido
El cuit 1234567890123 es invalido
El cuit CC-12345678-1 es invalido
Prueba pasada exitosamente!


In [20]:
class Factura:
    
    ultimoNumero = 100

    def __init__(self, catalogo, cliente): 
        self.catalogo = catalogo
        self.cliente = cliente
        self.numero = self.nuevoNumero()
        self.items = {}

    @staticmethod

    def ultimaFactura(numero):
        Factura._ultimaFactura = numero 

    @staticmethod
    def nuevoNumero():
        Factura.ultimoNumero += 1

        if Factura.ultimoNumero == 104: Factura.ultimoNumero = 10001

        return Factura.ultimoNumero

    def agregar(self, producto, cantidad):
        if producto.codigo in self.items:
            self.items[producto.codigo] += cantidad
        else:
            self.items[producto.codigo] = cantidad
        
        self.catalogo.vender(producto, cantidad)

        self._descuentos = self.descuentos
        self.total = self.subtotal - self._descuentos

    @property
    def cantidadProductos(self):
        return len(self.items)
    
    @property
    def cantidadUnidades(self):
        return sum(self.items.values())
    
    @property
    def subtotal(self):
        return sum(int(self.catalogo.buscar(codigo).precio) * cantidad for codigo, cantidad in self.items.items())
    
    @subtotal.setter
    def subtotal(self, value):
        self._subtotal = value

    @property

    def descuentos(self):
        total_descuento = 0
        for codigo, cantidad in self.items.items():
            producto = self.catalogo.buscar(codigo)
            total_descuento += self.catalogo.calcularDescuento(producto, cantidad)
        return total_descuento
    
    @property
    def total(self):
        return self.subtotal - self.descuentos
    
    def imprimir(self):
        detalles = []
        for codigo, cantidad in self.items.items():
            producto = self.catalogo.buscar(codigo)
            detalles.append(f"{cantidad} {producto.nombre}")
            descuento = self.catalogo.calcularDescuento(producto, cantidad)
            if descuento > 0:
                for oferta in self.catalogo.ofertas:
                    if oferta.esAplicable(producto, cantidad):
                        if isinstance(oferta, OfertaDescuento):
                           detalles.append(f"Descuento {int(oferta.descuento)}%")
                        elif isinstance(oferta, Oferta2x1):
                            detalles.append("Oferta 2x1")

        return (f"Cliente: {self.cliente.nombre}\n"
                f"CUIT: {self.cliente.cuit}\n"
                f"Productos:\n" + "\n".join(detalles) + "\n"
                f"Subtotal: {self.subtotal:.2f}\n"
                f"Descuentos: {self.descuentos:.2f}\n"
                f"TOTAL: {self.total:.2f}")
    @total.setter
    def total(self, value):
        self._total = value
    



In [21]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de la clase Factura #

# Creo un catálogo con productos
catalogo = Catalogo()
p1 = Producto('0001', 'Coca Cola',  1500, 'gaseosa', 10)
p2 = Producto('0002', 'Pepsi Cola', 1200, 'gaseosa', 20)
p3 = Producto('0003', 'Sonrisa',    1200, 'galleta', 30)
p4 = Producto('0004', 'Oreo',       2300, 'galleta', 40)
catalogo.agregar(p1,p2,p3,p4)

# Registro ofertas
catalogo.registrarOferta(Oferta2x1(tipos=['galleta']))
catalogo.registrarOferta(OfertaDescuento(10, codigos=['0001', '0003']))

# Creo un cliente
cliente = Cliente('Juan Perez', '20-12345678-9')

# Creo una factura
Factura.ultimaFactura(100)
assert Factura.nuevoNumero() == 101
assert Factura.nuevoNumero() == 102

f1 = Factura(catalogo, cliente)
f1.agregar(p1, 5)
f1.agregar(p3, 3)

assert f1.numero == 103
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades  == 8

# Agrega unidades de un producto ya agregado
f1.agregar(p1, 5)
assert f1.cantidadProductos == 2
assert f1.cantidadUnidades == 13

assert f1.subtotal   == 18600
assert f1.descuentos == 2700.0
assert f1.total == 15900.0

impresion = f1.imprimir()

assert "Juan Perez" in impresion
assert "10 Coca Cola" in impresion
assert "Sonrisa" in impresion
assert "Descuento 10%" in impresion
assert "Oferta 2x1" in impresion
assert "TOTAL:" in impresion
assert "15900.00" in impresion

print("Prueba pasada exitosamente!")

La cantidad 0 ingresada es invalida
Prueba pasada exitosamente!


In [22]:
## NO MODIFICAR ESTE CODIGO ##

# Prueba de integración #

# Cargamos los datos
catalogo = Catalogo.leer('catalogo.csv')
juan  = Cliente('Juan Perez', '20-12345678-9')
maria = Cliente('Maria Lopez', '27-87654321-3')

o2x1 = Oferta2x1(tipos=['galleta'], codigos=['0002', '0003','0010'])
od20 = OfertaDescuento(20, codigos=['0001', '0002'], tipos=['gaseosa', 'arroz'])
od10 = OfertaDescuento(10, tipos=['fideo'])

catalogo.registrarOferta(o2x1)
catalogo.registrarOferta(od20)
catalogo.registrarOferta(od10)

# Controlo que la carga este correcta
assert catalogo.cantidadProductos == 30
assert catalogo.cantidadUnidades == 1000
assert catalogo.valorTotal == 2000000


Factura.ultimaFactura(10000)

# Crear una factura
f1 = Factura(catalogo, juan)
f1.agregar(catalogo.buscar('0001'), 5)
f1.agregar(catalogo.buscar('0002'), 3)
f1.agregar(catalogo.buscar('0003'), 2)

assert f1.numero == 10001
assert f1.cantidadProductos == 3
assert f1.cantidadUnidades == 10
assert f1.subtotal == 13450.0
assert f1.descuentos == 3890.0
assert f1.total == 9560.0

assert catalogo.cantidadUnidades == 990

# Crear otra factura
f2 = Factura(catalogo, maria)
f2.agregar(catalogo.buscar('0010'), 5)
f2.agregar(catalogo.buscar('0010'), 3)
f2.agregar(catalogo.buscar('0020'), 2)
f2.agregar(catalogo.buscar('0030'), 2)

assert f2.numero == 10002
assert f2.cantidadProductos == 3
assert f2.cantidadUnidades == 12
assert f2.subtotal == 23900.00
assert f2.descuentos == 8860.00
assert f2.total == 15040.00

assert catalogo.cantidadUnidades == 978

print("Prueba pasada exitosamente!")

Prueba pasada exitosamente!
